Using kernels in GPflow
--

*James Hensman  2016*

GPflow comes with a range of kernels that can be combined to make new kernels. In this notebook, we examine some of the kernels, show how kernels can be combined, discuss the active_dims feature and show how one can build a new kernel. 

In [2]:
import GPflow
import numpy as np
import matplotlib
plt = matplotlib.pyplot
plt.style.use('ggplot')
import tensorflow as tf

### Kernel choices
First let's have a look at some of the standard kernels in GPflow. 

### Combining kernels


### Active dimesnsions


### Making new kernels
